In [1]:
import numpy as np
import random
from cmath import sqrt
from math import log
from functools import reduce

In [2]:
zero = np.matrix('1;0')
one = np.matrix('0;1')

In [3]:
def kron(*matrices):
    assert len(matrices) > 0, "Cannot perform Kronecker product on no matrices"
    return reduce(np.kron, matrices[1:], matrices[0])

In [4]:
class Q(object):

    def __init__(self, state):
        state_as_row = state.flatten().tolist()[0]
        sq_mag = sum(abs(a)**2 for a in state_as_row)
        assert abs(1-sq_mag) < 0.00001, "Squared magnitudes must sum to 1"
        self.state = state

    def __add__(self, other):
        new_state = self.state + other.state
        as_row = new_state.flatten().tolist()[0]
        norm_factor = sum(p**2 for p in as_row)
        norm_new_state = 1/norm_factor * new_state
        return Q(norm_new_state)
    
    def apply_gate(self, gate):
        new_state = gate * self.state
        as_row = new_state.flatten().tolist()[0]
        norm_factor = sum(p**2 for p in as_row)
        norm_new_state = 1/norm_factor * new_state
        return Q(norm_new_state)
    
    @property
    def num_qubits(self):
        rows, cols = self.state.shape
        qubits = log(rows, 2)
        assert qubits % 1.0 == 0, "Got irregular number of qubits"
        return int(qubits)
        
    def measure(self):
        amplitudes = self.state.flatten().tolist()[0]
        probabilities = ((a**2).real for a in amplitudes)
        cumul = 0
        rand = random.random()
        for idx, pdensity in enumerate(probabilities):
            cumul += pdensity
            if rand <= cumul:
                return idx
        raise AssertionError("probabilities did not sum to 1")
    
    def __repr__(self):
        return str(self.state)

In [5]:
I = np.eye(2)
X = np.matrix('0 1; 1 0')
Z = np.matrix('1 0; 0 -1')
H = 1/sqrt(2) * (X + Z)
entangled_pair = 1/sqrt(2) * (kron(zero, zero) + kron(one, one))

cnot_10 = np.matrix('1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0')
cnot_01 = np.matrix('1 0 0 0; 0 0 0 1; 0 0 1 0; 0 1 0 0')

def get_entangled_pair():
    return Q(entangled_pair)

def gate(qubits, unitary, qidx):
    gate_seq = (unitary if idx == qidx else I for idx in range(qubits))
    return kron(*gate_seq)

In [6]:
# 1. start with an entangled pair. Alice owns qubit 0 and Bob owns qubit 1
state = get_entangled_pair()
print("1. start with an entangled pair.")

# 2. Alice wants to send ab = 10
a, b = 1, 0

# If a = 1, Alice applies Z to her qubit.
if a == 1:
    state = state.apply_gate(gate(2, Z, 0))
# If b = 1, Alice applies X to her qubit.
if b == 1:
    state = state.apply_gate(gate(2, X, 0))
print("\n2. Alice wants to send ab = {}{}".format(a,b))

# 3. Alice sends her qubit to Bob.
print(state)

# 4. Bob applies CNOT with Alice's qubit as control.
# 2 qubit gate, qubit 0 is control, qubit 1 is target
state = state.apply_gate(cnot_10)
print("\n4. Bob applies CNOT with Alice's qubit as control.")

# 5. Bob applies H to Alice's qubit.
alice_h = gate(2, H, 0)
state = state.apply_gate(alice_h)
print("\n5. Bob applies H to Alice's qubit.")
print(state)

# 6. Bob measures the qubits to get message ab.
print("\n6. Bob measures the qubits to get message ab.")
measured = state.measure()
print(measured)

1. start with an entangled pair.

2. Alice wants to send ab = 10
[[ 0.70710678+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [-0.70710678+0.j]]

4. Bob applies CNOT with Alice's qubit as control.

5. Bob applies H to Alice's qubit.
[[ -2.23711432e-17+0.j]
 [  0.00000000e+00+0.j]
 [  1.00000000e+00+0.j]
 [  0.00000000e+00+0.j]]

6. Bob measures the qubits to get message ab.
2
